In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from spn.structure.leaves.parametric.Parametric import Categorical, Gaussian
from spn.algorithms.LearningWrappers import learn_parametric
from spn.structure.Base import Context

import pandas as pd
import numpy as np


bunch = fetch_openml(data_id = 40975)
bunch.frame

,buying,maint,doors,persons,lug_boot,safety,class
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5more,more,med,med,good
1724,low,low,5more,more,med,high,vgood
1725,low,low,5more,more,big,low,unacc
1726,low,low,5more,more,big,med,good


In [2]:
frame = bunch.frame.dropna()
names = frame.columns.tolist()
categories = list(names)
categorical = [True]*len(names)
cols = []
value_text = []
for name in names:
    if name in categories:
        values, text = pd.factorize(frame[name], sort=True)
        value_text.append(text.tolist())
        cols.append(values)
    else:
        cols.append(frame[name])
        value_text.append(None)

data = np.stack(cols, axis=1)
ptypes = [Categorical if name in categories else Gaussian for name in names]

train, test = train_test_split(data, random_state = 0)
print ("%d in train, %d in test" % (len(train), len(test)))

1296 in train, 432 in test


In [3]:
net = learn_parametric(train, 
                       ds_context = Context(parametric_types=ptypes).add_domains(train), 
                       rows = "gmm", ohe = True,
                       min_instances_slice = len(train) / 100)

/home/saurabh/miniconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [4]:
import csi2
from spn.structure.Base import get_nodes_by_type, Product



def format_condition(condition):
    try:
        a, sign, b = condition.split(" ")
    except ValueError as e:
        raise ValueError("Some nodes don't have conditions. reduce min_impurity_decrease")
    
    assert sign in ('==', '!=')
    i = names.index(a)
    v = value_text[i][int(b)]
    return ("%s %s %s" % (a, sign, v))


print ('#product nodes = ', len(get_nodes_by_type(net, (Product))))

csi2.annotate_spn(net, names, categorical, 
             min_impurity_decrease = 0.05, max_depth = 2)
rules = csi2.context_specific_independences(net, instance_threshold = 0)


rules = csi2.context_specific_independences(net, instance_threshold = 0)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

rules = csi2.context_specific_independences(net, instance_threshold = len(train) / 20,
                                           precision_threshold = 0.7, recall_threshold = 0.7)
csis = []
ac = []
cc = []
for i, rule in enumerate(rules):
    antecedent, consequent, *scores = rule
    A = csi2.format_antecedent(antecedent, format_condition)
    C = csi2.format_consequent(consequent)
    a_count = csi2.antecedent_count(A)
    c_count = csi2.consequent_count(C)
    ac.append(a_count)
    cc.append(c_count)
    csis.append ("{%s} => {%s} | %.2f, %.2f, %d | %d %d" % (A, C, *scores, a_count, c_count))
print ("%d, %.2f, %.2f" % (len(csis), np.mean(ac), np.mean(cc)))

#product nodes =  18
18, 5.22, 2.50
14, 5.21, 2.64


In [5]:
csis

['{((doors != 4) & (doors == 5more)) | (doors == 4)} => {(buying,persons,safety,class), (maint), (doors), (lug_boot)} | 1.00, 1.00, 655 | 3 4',
 '{(doors != 4) & (doors != 5more)} => {(buying,persons,safety,class), (maint), (doors), (lug_boot)} | 1.00, 1.00, 641 | 2 4',
 '{[((doors != 4) & (doors == 5more)) | (doors == 4)] & [((buying != high) & (buying == vhigh)) | (buying == high)]} => {(buying), (persons,safety,class)} | 1.00, 1.00, 338 | 6 2',
 '{[((doors != 4) & (doors == 5more)) | (doors == 4)] & [(buying != high) & (buying != vhigh)]} => {(buying), (persons,safety,class)} | 1.00, 1.00, 317 | 5 2',
 '{[(doors != 4) & (doors != 5more)] & [class != unacc]} => {(buying,safety,class), (persons)} | 1.00, 1.00, 175 | 3 2',
 '{[(doors != 4) & (doors != 5more)] & [class == unacc]} => {(class), (buying), (persons), (safety)} | 1.00, 1.00, 466 | 3 4',
 '{[((doors != 4) & (doors == 5more)) | (doors == 4)] & [((buying != high) & (buying == vhigh)) | (buying == high)] & [class != acc]} => {(c

In [6]:
from spn.algorithms.Inference import log_likelihood

print ("%.2f" % np.mean(log_likelihood(net, test)))

-7.92
